In [ ]:
import pandas as pd
!pip install tweet-preprocessor
import preprocessor as p
import numpy as np
from tqdm.notebook import tqdm_notebook, tqdm
import requests
from io import StringIO
from google.colab import drive
import string

import sys
import csv
maxInt = sys.maxsize

while True:
    # decrease the maxInt value by factor 10 
    # as long as the OverflowError occurs.

    try:
        csv.field_size_limit(maxInt)
        break
    except OverflowError:
        maxInt = int(maxInt/10)

drive.mount('/content/drive')
test_path = '/content/drive/My Drive/UK/development/UK_bal_test_indiv.csv'
train = pd.read_csv('/content/drive/My Drive/UK/development/train_indiv.csv', index_col=0, encoding='utf8', engine='python', error_bad_lines=False)
train= train.sample(frac=1)[:5000]
import nltk
from nltk.corpus import gutenberg, stopwords 
nltk.download('stopwords')
stop_words = set(stopwords.words('english')) 

print(np.shape(train.values))
w = train.Label.value_counts(normalize=True)
weights ={0:1/w[0], 1:1/w[1]}
print(weights)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
(5000, 4)
{0: 1.0895619960775769, 1: 12.165450121654501}


# Preprocessing

Cleanining of text samples

In [ ]:
p.set_options(p.OPT.URL, p.OPT.MENTION, p.OPT.HASHTAG)
min_occurance =10
vocab_freq = {'<UNK>':min_occurance}

for tweet in tqdm(train.CTweet.values.tolist()):
  #tweet=(((((tweet.replace(',', '')).replace('.', '')).replace('?', '')).replace('!', '')).replace('[', '')).replace(']', '')
  tokenized = p.tokenize(tweet).split(' ')
  for word in tokenized:
    if word not in vocab_freq.keys():
      vocab_freq[word] = 1
    else:
      vocab_freq[word]+=1

def input_preprocess(df):
  recons_sent = []
  for tweet in tqdm(df.CTweet.values.tolist()):
    #tweet=(((((tweet.replace(',', '')).replace('.', '')).replace('?', '')).replace('!', '')).replace('[', '')).replace(']', '')
    tokenized = p.tokenize(tweet).split(' ')
    new_sent = []
    for word in tokenized:
      if word not in vocab_freq.keys() or vocab_freq[word] <min_occurance-1 or word in stop_words or word.isdigit():
        new_sent.append('<UNK>')
      else:
        new_sent.append(word)
    recons_sent.append(' '.join(new_sent))
  return recons_sent
train_input = input_preprocess(train)

In [ ]:
final_vocab =[key for key, val in vocab_freq.items() if val >=min_occurance-1]
print(len(final_vocab))
train

940


,Date,Username,CTweet,Label
2709356,2014-11-27 18:21:32+00:00,missgeorgieanne,<S> looking forward to meeting on monday for s...,None
2430455,2019-02-15 21:00:57+00:00,DanielPryorr,<S> can i get uhh a umm glass of freedom water...,None
2219898,2019-10-19 21:23:15+00:00,MissKittenFilan,<S> awww im xx <S/>,Depressed
83161,2016-09-11 19:17:05+00:00,jessdwi,<S> wonder how long itll take the fa to banfin...,None
951224,2019-08-02 19:04:32+00:00,spencer_sian,<S> gardening wood walk with amydog ( who is a...,None
...,...,...,...,...
2648848,2019-11-23 16:32:06+00:00,speelingmstake,<S> oh boy the one after that though q my whol...,None
2581487,2019-08-18 16:51:55+00:00,ChrisMears1,<S> with tommy baldwin hes alive and well <S/>,None
1568479,2018-07-18 10:34:49+00:00,MattNealRacing,<S> good luck this weekend eyes on the prize b...,None
2481991,2014-07-22 12:17:17+00:00,Cloutos,<S> but first let me take a selfie worse moon ...,None


Labels tokenisation

In [ ]:
from keras.utils import to_categorical
def labels(df, t = None):
  lab = []
  if t == 'time':
    for l in df['Label'].values:
      if 'D' in list(l):
        lab.append(1)
      else:
        lab.append(0)
    return (np.array(lab)).astype(int)

  else:
    if 'Depressed' in df.Label.values.tolist():
      print('inside')
    #return pd.get_dummies(df.Label).reset_index(drop=True)
    
    for i in df.Label:
      if i =='Depressed':
        lab.append(1)
      else:
        lab.append(0)
    return (np.array(lab)).astype(int)

train_labels = labels(train, t ='time')



In [ ]:
test = pd.read_csv(test_path, index_col=0, encoding='utf8', engine='python', error_bad_lines=False)[:2500]
test_labels =labels(test, t='time')

test_input = input_preprocess(test)


#del train, stopwords#, vocab_freq, final_vocab

# Classification

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report
cv = CountVectorizer()

from sklearn.svm import LinearSVC
print(list(weights.values()))
new_weights = {0:list(weights.values())[0], 1:list(weights.values())[1]}

svm = LinearSVC(class_weight=new_weights)#class_weight=new_weights)#class_weight=new_weights)#class_weight=new_weights)
svm.fit((cv.fit_transform(train_input)).toarray(), train_labels)


predictions = svm.predict((cv.transform(test_input)))

print(classification_report(test_labels, predictions, target_names=['None', 'Depressed']))


[1.0895619960775769, 12.165450121654501]
{0: 1.0895619960775769, 1: 12.165450121654501}
started training
done training
done predicting
              precision    recall  f1-score   support

        None       0.54      0.72      0.62      1291
   Depressed       0.54      0.35      0.43      1209

    accuracy                           0.54      2500
   macro avg       0.54      0.54      0.52      2500
weighted avg       0.54      0.54      0.53      2500



/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


"test['Label'] =pd.Series(test_labels, index=test.index)\ntest= test.dropna()\ntest_sum = test.groupby('Username').sum()\ndel test\ntest_sum\n\ntp=0\ntn=0\nfn=0\nfp=0\nfor label, pred in zip(test_sum['Label'], test_sum['SVM']):\n    if label == 0:\n        #Negative\n        if pred ==0:\n            tn+=1\n        elif pred>0:\n            fp+=1\n    elif label >0:\n        if pred>0:\n            tp+=1\n        elif pred==0:\n            fn+=1\n\nprint('True Positive: {}, True Negative: {}, False Positive: {}, False Negative: {}'.format(tp, tn, fp, fn))\npre = tp/(tp+fp)\nr=tp/(tp+fn)\nprint('Precision: {}'.format(pre))\nprint('Recall: {}'.format(r))\nprint('Fscore: {}'.format(2*(pre*r)/(pre+r)))"